In [1]:
##Simple Gen AI App using LangChain and OpenAI

import os
from dotenv import load_dotenv
load_dotenv()


os.environ["openai_api_key"] = os.getenv("OPENAI_API_KEY")
os.environ["langchain_api_key"] = os.getenv("LANGCHAIN_API_KEY")
os.environ["langchain_tracing_v2"] = "true"
os.environ["langchain_project"] = os.getenv("LANGCHAIN_PROJECT", "GenAIappWithOPENAI")

In [5]:
##Data injestion
from langchain_community.document_loaders import WebBaseLoader
loader = WebBaseLoader("https://docs.smith.langchain.com/administration/concepts")
docs=loader.load()
print(docs[0].page_content[:500])  # Print the first 500 characters of the first document






Concepts | 🦜️🛠️ LangSmith








Skip to main contentOur Building Ambient Agents with LangGraph course is now available on LangChain Academy!API ReferenceRESTPythonJS/TSSearchRegionUSEUGo to AppGet StartedObservabilityEvaluationPrompt EngineeringDeployment (LangGraph Platform)AdministrationTutorialsOptimize tracing spend on LangSmithHow-to GuidesSetupConceptual GuideSelf-hostingPricingReferenceCloud architecture and scalabilityAuthz and AuthnAuthentication methodsdata_formatsEvaluationData


In [20]:
##load data to chunk        
from langchain.text_splitter import RecursiveCharacterTextSplitter
recursive_text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,chunk_overlap=200)

docs = recursive_text_splitter.split_documents(docs)
print(docs[0].page_content[:5000])  # Print the first 500 characters

Concepts | 🦜️🛠️ LangSmith


In [21]:
from langchain_ollama import OllamaEmbeddings
embeddings = OllamaEmbeddings(model="llama2")



In [22]:
from langchain_community.vectorstores import FAISS
db = FAISS.from_documents(docs, embeddings)

In [23]:
db

In [ ]:
query = "What is the purpose of LangSmith?"
docs = db.similarity_search(query, k=3)
docs[0].page_content  # Print the first 500 characters of the first document

'Feedback is added to any run on the trace\nAn Annotation Queue receives any run from the trace\nA Run Rule matches any run within a trace\n\nWhy auto-upgrade traces?\nWe have two reasons behind the auto-upgrade model for tracing:\n\nWe think that traces that match any of these conditions are fundamentally more interesting than other traces, and\ntherefore it is good for users to be able to keep them around longer.\nWe philosophically want to charge customers an order of magnitude lower for traces that may not be interacted with meaningfully.\nWe think auto-upgrades align our pricing model with the value that LangSmith brings, where only traces with meaningful interaction\nare charged at a higher rate.\n\nIf you have questions or concerns about our pricing model, please feel free to reach out to support@langchain.dev and let us know your thoughts!\nHow does data retention affect downstream features?'

In [30]:
from langchain_ollama import OllamaLLM
llm = OllamaLLM(model="llama2")

In [34]:
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

# Create a proper chat prompt template
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant."),
    ("user", "Answer the following question based only on the provided context. If you don't know the answer, just say that you don't know, don't try to make up an answer.\n\nContext: {context}")
])

# Create a document chain using your LLM and the prompt
document_chain = create_stuff_documents_chain(llm, prompt)

# Optional: combine with another component (if desired)
# Here you're chaining the document_chain directly with the LLM, which is unnecessary if create_stuff_documents_chain already handles it.
# You can just use document_chain directly like this:


In [37]:
from langchain_core.documents import Document
response = document_chain.invoke({"context": docs, "question": "What is the purpose of LangSmith?"})
print(response)

The auto-upgrade traces feature is implemented to enhance the user experience and provide better value for customers. By automatically upgrading traces that match certain conditions, such as having more interesting or meaningful interactions, LangSmith can offer a pricing model that aligns with the value it brings to its users. This means that traces that are more relevant or engaging to users will be charged at a lower rate, while traces with less meaningful interaction will be charged at a higher rate.

Data retention, on the other hand, affects downstream features by determining how long tracing data is kept available for analysis and use. A longer data retention period allows users to access and analyze more historical data, which can provide valuable insights into trends and patterns over time. This can be particularly useful for tasks such as:

* Identifying seasonal or long-term trends in user behavior
* Tracking the effectiveness of marketing campaigns over time
* Monitoring ch

In [41]:
vectorstore = db.as_retriever()
from langchain.chains import create_retrieval_chain
retrieval_chain = create_retrieval_chain(vectorstore, document_chain)

response = retrieval_chain.invoke({"input": "What is the purpose of LangSmith?"})
print(response)

{'input': 'What is the purpose of LangSmith?', 'context': [Document(id='e6408e29-f7e6-475c-8fca-421bd4414335', metadata={'source': 'https://docs.smith.langchain.com/administration/concepts', 'title': 'Concepts | 🦜️🛠️ LangSmith', 'description': 'This conceptual guide covers topics related to managing users, organizations, and workspaces within LangSmith.', 'language': 'en'}, page_content='Feedback is added to any run on the trace\nAn Annotation Queue receives any run from the trace\nA Run Rule matches any run within a trace\n\nWhy auto-upgrade traces?\nWe have two reasons behind the auto-upgrade model for tracing:\n\nWe think that traces that match any of these conditions are fundamentally more interesting than other traces, and\ntherefore it is good for users to be able to keep them around longer.\nWe philosophically want to charge customers an order of magnitude lower for traces that may not be interacted with meaningfully.\nWe think auto-upgrades align our pricing model with the valu